In [38]:
import pandas as pd
import sys
import numpy as np
from sklearn.datasets import load_svmlight_file, dump_svmlight_file


In [39]:




#load data
print ('loading \'iris\' (multiclass) dataset...', end='')
from sklearn.datasets import load_iris
ds = load_iris()
X,Y = ds.data, ds.target

'''
WARNING: be sure that your matrix is not sparse! EXAMPLE:
from sklearn.datasets import load_svmlight_file
X,Y = load_svmlight_file(...)
X = X.toarray()
'''


loading 'iris' (multiclass) dataset...done
0.001766204833984375


In [40]:
#preprocess data
print ('preprocessing data...', end='')
#boolean kernels can be applied on binary-valued data, i.e. {0,1}.
#in this example, we binarize a real-valued dataset
from MKLpy.preprocessing import binarization
binarizer = binarization.AverageBinarizer()
Xbin = binarizer.fit_transform(X,Y)
print ('done')

#compute normalized homogeneous polynomial kernels with degrees 0,1,2,...,10.
print ('computing monotone Conjunctive Kernels...', end='')
from MKLpy.metrics import pairwise
from MKLpy.preprocessing import kernel_normalization
#WARNING: the maximum arity of the conjunctive kernel depends on the number of active variables for each example,
# that is 4 in the case of iris dataset binarized
KL = [kernel_normalization(pairwise.monotone_conjunctive_kernel(Xbin, c=c)) for c in range(5)]
print ('done')

preprocessing data...done
computing monotone Conjunctive Kernels...done


In [41]:
#train/test KL split (N.B. here we split a kernel list directly)
from MKLpy.model_selection import train_test_split
KLtr,KLte,Ytr,Yte = train_test_split(KL, Y, test_size=.3, random_state=42)

In [36]:

#MKL algorithms
from MKLpy.algorithms import AverageMKL, EasyMKL, KOMD	#KOMD is not a MKL algorithm but a simple kernel machine like the SVM
print ('training AverageMKL...', end='')
clf = AverageMKL().fit(KLtr,Ytr)#a wrapper for averaging kernels
/

# print ('training EasyMKL...', end='')
# clf = EasyMKL(lam=0.1).fit(KLtr,Ytr)		#combining kernels with the EasyMKL algorithm
# #lam is a hyper-parameter in [0,1]
# print ('done')
# print (clf.weights)

# #evaluate the solution
# from sklearn.metrics import accuracy_score, roc_auc_score
# y_pred = clf.predict(KLte)					#predictions
# y_score = clf.decision_function(KLte)		#rank
# accuracy = accuracy_score(Yte, y_pred)
# roc_auc = roc_auc_score(Yte, y_score)
# print ('Accuracy score: %.3f, roc AUC score: %.3f' % (accuracy, roc_auc))


# #select the base-learner
# #MKL algorithms use a hard-margin as base learned (or KOMD in the case of EasyMKL). It is possible to define a different base learner
# from sklearn.svm import SVC
# base_learner = SVC(C=0.1)
# clf = EasyMKL(estimator=base_learner)
# clf = clf.fit(KLtr,Ytr)

training AverageMKL...

AttributeError: 'AverageMKL' object has no attribute 'estimator'

In [37]:
MKLpy.algorithms.AverageMKL?

In [26]:
#preprocess data
print ('preprocessing data...', end='')
from MKLpy.preprocessing import normalization, rescale_01
X = rescale_01(X)	#feature scaling in [0,1]
X = normalization(X) #||X_i||_2^2 = 1

#train/test split
from sklearn.model_selection import train_test_split
Xtr,Xte,Ytr,Yte = train_test_split(X,Y, test_size=.25, random_state=42)
print ('done')


#compute homogeneous polynomial kernels with degrees 0,1,2,...,10.
print ('computing Homogeneous Polynomial Kernels...', end='')
from MKLpy.metrics import pairwise
KLtr = [pairwise.homogeneous_polynomial_kernel(Xtr, degree=d) for d in range(11)]
KLte = [pairwise.homogeneous_polynomial_kernel(Xte,Xtr, degree=d) for d in range(11)]
print ('done')


preprocessing data...done
computing Homogeneous Polynomial Kernels...done


In [35]:
#evaluation on the test set
from sklearn.metrics import accuracy_score
print ('done')
clf = EasyMKL().fit(KLtr,Ytr)
y_pred = clf.predict(KLte)
accuracy = accuracy_score(Yte, y_pred)
print ('accuracy on the test set: %.3f, with lambda=%.2f' % (accuracy, best_results['lam']))

done


AttributeError: 'EasyMKL' object has no attribute 'max_iter'

In [12]:
from MKLpy.regularization import normalization,rescale_01
from MKLpy.metrics.pairwise import HPK_kernel
# from MKLpy.regularization import kernel_centering, kernel_normalization, tracenorm
# from MKLpy.algorithms import EasyMKL,RMGD,RMKL,AverageMKL

# from sklearn.metrics.pairwise import rbf_kernel as RBF

ModuleNotFoundError: No module named 'MKLpy.regularization'

In [ ]:
MKLpy.ipynb